In [24]:
#Dependencies
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import datetime as dt

## Reflect Tables into SQLAlchemy ORM

In [26]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
#from sqlalchemy import create_engine, text
from sqlalchemy import create_engine, func

In [27]:
# create engine to hawaii.sqlite
engine = create_engine(f"sqlite:///../Resources/hawaii.sqlite")

In [31]:
# reflect an existing database into a new model

# reflect the tables

In [30]:
# View all of the classes that automap found
Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()


['station', 'measurement']

In [33]:
# Save references to each table
h_measurement=Base.classes.measurement
h_station=Base.classes.station

In [35]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)

In [36]:
first_row=session.query(h_measurement).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x2db559fa690>,
 'tobs': 65.0,
 'date': '2010-01-01',
 'prcp': 0.08,
 'id': 1,
 'station': 'USC00519397'}